# Fresco Constructor

This code is used for constructing [FRESCO](http://www.fresco.org.uk/) coupled-channel reaction input files.

This code is only suitable for (d,p) reactions.

In [2]:
#imports
import numpy as np
import pandas as pd
import re
from dataclasses import dataclass

### Mass Data Imports

Here we import the [Atomic Mass Evaluation 2020](https://www-nds.iaea.org/amdc/) for use in the reaction codes.

In [4]:
#import NNDC Masses as a Pandas DataFrame
masses = pd.read_fwf("mass_1.mas20.txt",skiprows=33,infer_nrows=3500) 
#infer rows to almost the entire thing so that it parses the larger N,Z,A values correctly


In [5]:
#masses.head()

In [6]:
#drop some columns we don't care about
masses.drop(['1N-Z','O','MASS EXCESS           BINDING ENERGY/A        BETA-DECAY ENERGY'],axis=1,inplace=True)


In [7]:
#drop the units underneath the header keys
masses.drop([0],axis=0,inplace=True)

In [8]:
#masses.head()

In [9]:
#convert the split mass (in micro-amu) into a single micro-amu column
masses['Atomic Mass (micro-amu)'] = masses['ATOMIC MASS'].apply(lambda x: float(x.split("#")[0])) + masses['Unnamed: 7']*1e6
#the split("#") is needed to remove any of the "#" that appear due to estimated errors 

#if *'s are also present in the data:
#masses['Atomic Mass'] = masses['ATOMIC MASS'].apply(lambda x: float(re.split("# *",x)[0])) + masses['Unnamed: 7']*1e6


In [10]:
#create masses in MeV
masses['Atomic Mass (MeV)'] = masses['Atomic Mass (micro-amu)'] *931.5/1.e6 #931.5 MeV = 1 amu. Also convert from micro-amu to amu

In [11]:
masses.head()

N    Z    A  EL  Unnamed: 7    ATOMIC MASS Unnamed: 9  \
1  1.0  0.0  1.0   n         1.0   008664.91590    0.00047   
2  0.0  1.0  1.0   H         1.0  007825.031898   0.000014   
3  1.0  1.0  2.0   H         2.0  014101.777844   0.000015   
4  2.0  1.0  3.0   H         3.0   016049.28132    0.00008   
5  1.0  2.0  3.0  He         3.0   016029.32197    0.00006   

   Atomic Mass (micro-amu)  Atomic Mass (MeV)  
1             1.008665e+06         939.571369  
2             1.007825e+06         938.789017  
3             2.014102e+06        1876.135806  
4             3.016049e+06        2809.449906  
5             3.016029e+06        2809.431313

In [12]:
#all of the elements
atomicElements = "H He Li Be B C N O F Ne Na Mg Al Si P S Cl Ar K Ca Sc Ti V Cr Mn Fe Co Ni Cu Zn Ga Ge As Se Br Kr Rb Sr Y Zr Nb Mo Tc Ru Rh Pd Ag Cd In Sn Sb Te I Xe Cs Ba La Ce P Nd Pm Sm Eu Gd Tb Dy Ho Er Tm Yb Lu Hf Ta W Re Os Ir Pt Au Hg Tl Pb Bi Po At Rn Fr Ra Ac Th Pa U Np Pu Am Cm Bk Cf Es Fm Md No Lr Rf Db Sg Bh Hs Mt Ds Rg Cn Nh Fl Mc Lv Ts Og".split(" ")


In [13]:
#function for getting the L value of an orbital
def get_orbital_L(orbital):
    orbital_indexes = possible_orbitals = "s p d f g h i j k l".split(" ")
    try:
        L_value = orbital_indexes.index(orbital)
    except:
        return -1
    else:
        return L_value

In [14]:
#determine transferred particle function
def calculate_transferred_particle(beam,recoil,mass_df):
    A = abs(beam.A - recoil.A)
    Z = abs(beam.Z - recoil.Z)
    #check if we have a neutron:
    if (0 == Z):
        transferred_particle = Isotope(A,"n",mass_df) 
    else:
        transferred_particle = Isotope(A,atomicElements[Z-1],mass_df) #-1 because we index at 0

    return transferred_particle

In [15]:
#class for an isotope
#this will hold A,Z,El,mass, etc.
#@dataclass
class Isotope:

    def __init__(self,A,element,mass_df): #,spin = 0,parity=0,exc_energy=0,q_value = 0,L_transfer = 0,orbital_osc_shell = 0,orbital_orbital = 's',orbital_L=0,orbital_J=0,nn = 0,binding_energy = 0):
        self.A = A
        self.element = element

        #fill in these automatically
        self.get_proton_number()
        self.get_mass(mass_df)
        self.get_mass_amu()
        self.get_isotope()
        
        #fill in these with default 0 for everything
        self.spin = 0
        self.parity = 0
        self.exc_energy = 0
        self.q_value = 0
        self.L_transfer = 0 #L transfer to this state (in the case of this being the recoil)
        #associated orbital info for a recoil nucleus only
        self.orbital_osc_shell = 0 #major oscillator shell of orbital being transferred into
        self.orbital_orbital = 0 #orbital name (s, p, d, f, etc) of orbital being transferred into
        self.orbital_L = 's' #angular momentum associated with the orbital being transferred into
        self.orbital_J = 0
        self.nn = 0 #number of nodes
        self.binding_energy = 0
        

    
    
    #automatic calculations and traits here
    def get_proton_number(self):
        if "n" == self.element:
            self.Z = 0
        else:
            self.Z = atomicElements.index(self.element)+1
        
    def get_mass(self,mass_df):
        try:
            mass = float(mass_df[(mass_df['EL']== self.element) & (mass_df['A'] == self.A)]['Atomic Mass (MeV)'].iloc[0])
        except:
            self.mass = 0
        else:
            self.mass = mass
    def get_mass_amu(self):
        try:
            mass_amu = self.mass / 931.5
        except:
            self.mass_amu = 0
        else:
            self.mass_amu = mass_amu

    def get_isotope(self):
        self.isotope = str(self.A)+str(self.element)


    
    def __str__(self):
        return f'''Isotope: {self.isotope}\nA: {self.A}\nElement: {self.element}\nZ: {self.Z}\nMass (MeV): {round(self.mass,3)}
        \rMass (amu): {round(self.mass_amu,3)}\nJ: {self.spin}\nParity: {self.parity}\nExc. energy (MeV): {self.exc_energy}
        \rQ-value (MeV): {self.q_value}\nL-transfer: {self.L_transfer}\nOrbital shell: {self.orbital_osc_shell}
        \rOrbital orbital: {self.orbital_orbital}\nOrbital L: {self.orbital_L}\nOrbital J: {self.orbital_J}\nNn: {self.nn}
        \rBinding energy (MeV): {round(self.binding_energy,3)}'''
    

In [16]:
iso = Isotope(88,'Kr',masses)
print(iso)

Isotope: 88Kr
A: 88
Element: Kr
Z: 36
Mass (MeV): 81892.308
Mass (amu): 87.914
J: 0
Parity: 0
Exc. energy (MeV): 0
Q-value (MeV): 0
L-transfer: 0
Orbital shell: 0
Orbital orbital: 0
Orbital L: s
Orbital J: 0
Nn: 0
Binding energy (MeV): 0


In [17]:
#iso = Isotope(87,'Kr',0,-1)
#print(iso)

In [18]:

#function for getting user input for 
def get_user_isotope(mass_df):

    my_iso = None
    while True:
        element = "BAD"
        mass_number = -1
        
        #get the element
        while element not in atomicElements:
            element = input("Please enter the name (ex: Kr):" )
        
        #get the mass number
        while mass_number not in range(1,300):
            try:
                mass_number = int(input("Please enter the mass number: "))
            except:
                print("Looks like you did not enter an integer!")
                
            else:
                if (mass_number not in range(1,300)):
                    print("Mass number out of range!\nMust have mass 1-300")
                    
        #create our isotope    
        my_iso = Isotope(mass_number,element,mass_df)
        if (0 == my_iso.mass):
            print(f"Isotope {my_iso.isotope} does not exist in the AME2020. Try again!")
        else:
            break
    
        #print(f'Selected isotope: {mass_number}{element}, mass {round(isotope_mass,3)} MeV')

    #now return this A,El, mass as a tuple
    #return (mass_number,element,isotope_mass)
    return my_iso


In [19]:
#kr93 = get_user_isotope()
#kr93

In [20]:
#get the beam energy

def get_user_beam_energy():

    beam_energy = -1
    
    #get the beam energy
    while beam_energy < 0:
        
        try:
            beam_energy = float(input("Enter total beam energy (MeV): "))
        except:
            print("Beam energy is not a number (>0). Try again")
            #continue
        else:
            if (beam_energy > 0):
                return beam_energy
    

In [21]:
#get_user_beam_energy()

In [22]:
#get_proton_number('Og')

In [23]:
#initial setup of the FRESCO header info. This is mostly unchanged from reaction to reaction
def construct_fresco_header(lab_energy):

    file = open('build_template_files/NAMELIST.txt','r')
    namelist_header = file.read()
    namelist_header = namelist_header.replace("LAB_ENERGY",str(lab_energy)) #lab energy in MeV
        
    #print(namelist_header)

    return namelist_header


In [24]:
#partition function

def construct_partition(beam,target,recoil,num_states): #beam is a tuple of A,El,mass
    file = open('build_template_files/PARTITION.txt','r')
    partition_info = file.read()

    #unpack tuples
    #A,El,mass = beam
    #A_targ, El_targ, mass_targ = target

    #reassign parts
    partition_info = partition_info.replace('BEAM_ISOTOPE',"' "+beam.isotope+"'")
    #print(type(mass))
    partition_info = partition_info.replace('BEAM_MASS',str(round(float(beam.mass_amu),3)))
    partition_info = partition_info.replace('BEAM_Z', str(beam.Z)) #plus one because we index at 0, but periodic table indexes as t
    partition_info = partition_info.replace('TARGET_NAME',"' "+target.isotope+"'")
    partition_info = partition_info.replace('TARGET_MASS',str(round(float(target.mass_amu),3)))
    partition_info = partition_info.replace('TARGET_Z',str(target.Z)) #plus one because of indexing again
    partition_info = partition_info.replace('Q_VALUE',str(round(recoil.q_value,3)))
    partition_info = partition_info.replace('NUM_STATES',str(num_states))
    
    #print(partition_info)
    return partition_info




In [25]:
#construct_partition(('A','Kr','13.23423432'),('B','H','32.323242342'),('1.34','5'))

In [26]:
#state function
def construct_states(beam,target,recoil):
    file = open('build_template_files/STATE.txt','r')
    state_info = file.read()


    state_info = state_info.replace('BEAM_J',str(beam.spin))
    state_info = state_info.replace('BEAM_PARITY',str(beam.parity))
    state_info = state_info.replace('BEAM_EXC_ENERGY',str(beam.exc_energy))
    state_info = state_info.replace('TARGET_J',str(target.spin))
    state_info = state_info.replace('TARGET_PARITY',str(target.parity))
    state_info = state_info.replace('Q_VALUE',str(round(recoil.q_value,3)))
    
    #print(state_info)
    return state_info


In [27]:
#overlap function
def construct_overlap(recoil,number):
    file = open('build_template_files/OVERLAP.txt')
    overlap_info = file.read()

    overlap_info = overlap_info.replace('STATE_NUM',number)
    overlap_info = overlap_info.replace('STATE_NN',str(recoil.nn))
    overlap_info = overlap_info.replace('STATE_L',str(recoil.orbital_L))
    overlap_info = overlap_info.replace('STATE_J',str(recoil.orbital_J))
    overlap_info = overlap_info.replace('STATE_SN',str(round(recoil.binding_energy - recoil.exc_energy,3)))

    #this is needed to change the potential used for the deuteron only
    if ("2H" == recoil.isotope):
        overlap_info = overlap_info.replace('ic1=2','ic1=1')
        overlap_info = overlap_info.replace('ic2=1','ic2=2')
        overlap_info = overlap_info.replace('in=1','in=2')
        overlap_info = overlap_info.replace('kbpot=5','kbpot=4')

    return overlap_info

In [28]:
#coupling header
def construct_coupling():
    file = open('build_template_files/COUPLING.txt')
    coupling_info = file.read()

    return coupling_info

In [29]:
#cfp info
def construct_cfp(state_num):
    file = open('build_template_files/CFP.txt')
    cfp_info = file.read()

    #if we have a deuteron, do something extra:
    if ("" == state_num):
        cfp_info = cfp_info.replace("in=1","in=2")
    
    cfp_info = cfp_info.replace('STATE_NUM',state_num)

    return cfp_info

In [30]:
#this will write the potential 
def construct_potential(isotope,potential,beam_energy,number):
    file = open('build_template_files/POTENTIAL.txt')
    potential_info = file.read()
    
    #replace the stuff we need now
    potential_info = potential_info.replace('MASS',str(isotope.A))
    potential_info = potential_info.replace('RC',str(potential.RC))
    potential_info = potential_info.replace('KP_NUM',str(number))
    potential_info = potential_info.replace('V0',str(round(potential.V0,4)))
    potential_info = potential_info.replace('R0',str(round(potential.R0,4)))
    potential_info = potential_info.replace('A0',str(round(potential.A0,4)))
    if (3 != number):#only do these if we aren't in the beam+neutron potential
        potential_info = potential_info.replace('WD',str(round(potential.WD,4)))
        potential_info = potential_info.replace('RD',str(round(potential.RD,4)))
        potential_info = potential_info.replace('AD',str(round(potential.AD,4)))
        potential_info = potential_info.replace('VL',str(round(potential.VLS,4)))
        potential_info = potential_info.replace('RL',str(round(potential.RLS,4)))
        potential_info = potential_info.replace('AL',str(round(potential.ALS,4)))

    return potential_info



In [31]:
#this constructs the deuteron binding potential. Not dependent on the beam
def construct_deuteron_potential():
    file = open('build_template_files/DEUTERON_POTENTIAL.txt')
    deut_info = file.read()

    return deut_info

In [32]:
#binding energy calculation
def calculate_binding_energy(recoil,bound_nucleus,mass_df):
    
    #first calculate A and Z/element for the core
    A = recoil.A - bound_nucleus.A
    Z = recoil.Z - bound_nucleus.Z
    element = atomicElements[Z-1]
    core = Isotope(A,element,mass_df)

    #binding energy calculation here
    BE = core.mass + bound_nucleus.mass - recoil.mass
    #print(BE)
    recoil.binding_energy = BE
    
    return

In [33]:
#binding energy of the deuteron specifically
def calculate_deuteron_binding_energy(mass_df):
    deut = Isotope(2,"H",mass_df)
    prot = Isotope(1,"H",mass_df)
    neut = Isotope(1,"n",mass_df)

    return  (prot.mass + neut.mass) - deut.mass

In [34]:
calculate_deuteron_binding_energy(masses)

2.2245803121513745

In [35]:
#get the spin-parity for the parent state
def get_user_spin_parity(my_isotope):

    #A,El,mass = beam
   # Z = get_proton_number(El) #find the z of our nucleus

    #if we have an even-even nucleus, return 0+
    #return the form (spin,parity) where parity is defined by +-1
    if (0 == my_isotope.A % 2) and (0 == my_isotope.Z % 2):
        print("Even-even nucleus. Using spin 0+")
        my_isotope.spin = 0
        my_isotope.parity = +1
        return

    #if we have a deuteron, we know what that is too:
    elif (2 == my_isotope.A) and ('H' == my_isotope.element):
        my_isotope.spin = 1
        my_isotope.parity = +1
        return 

    
    else:
        #get a spin that is an whole number of positive half-integer
        get_user_spin(my_isotope)

        
        #now get the parity:
        get_user_parity(my_isotope)

    return


#get spin function
def get_user_spin(isotope):
    #get a spin that is an whole number of positive half-integer
    possible_spins = np.linspace(0,20,41) #unlikely to have a spin higher than 20
    spin = -1
    
    while spin not in possible_spins:
        try:
            spin = float(input("Enter the spin: "))
        except:
            print("Spin was not a number!")
            #continue
        else:
            if (spin not in possible_spins):
                print("Spin is not an non-negative integer or half-integer!")
            else:
                isotope.spin = spin
    return


#get parity function
def get_user_parity(isotope):
    #get the parity:
    parity = 0
    while parity not in (-1,1):
        try:
            parity = int(input(u"Please enter the parity (\u00B11): "))
        except:
            print("Parity is not a number!")
        else:
            if parity not in (-1,1):
                print(u"Parity not in \u00B11")
            else:
                isotope.parity = parity
    return

In [36]:
#function for the number of states
def get_user_number_states():
    num_states = -1
    while num_states < 1:
        try:
            num_states = int(input("Enter the number of states to populate: "))
        except:
            print("Number of states must be a natural number!")
        else:
            if (num_states > 0):
                return num_states

In [37]:
#get_user_number_states()

In [38]:


#get the energy itself
def get_user_excited_state_energy(isotope):

    energy  = -1
    while energy < 0:
        try:
            energy = float(input("Enter the excited-state energy (MeV): "))
        except:
            print("Energy must be a non-negative number!")
        else:
            if (energy >= 0):
                isotope.exc_energy = energy

    return
    


In [39]:
#function for getting the associated orbital for the transfer
def get_user_orbital(isotope):
    orbital_string = "BAD"
    orbital = "BAD"

    possible_orbitals = "s p d f g h i j k l".split(" ")
    while orbital not in possible_orbitals:
        orbital_string = input("Enter the orbital (ex: 2d5/2): ")
        #split the orbital string and find just the orbital and oscillator
        try:
            shell, orbital, J = re.split(r'([a-z])',orbital_string)
        except:
            print("Not a valid orbital.")
        else:
            if orbital not in possible_orbitals:
                print("Not a valid orbital.")
            else:
                isotope.orbital_osc_shell = int(shell)
                isotope.orbital_orbital = orbital
                
                isotope.orbital_L = get_orbital_L(orbital)
                num, denom = J.split('/')
                isotope.orbital_J = float(num)/float(denom)
            #    print(shell)
            #    print(orbital)
            #    print(L)
    return



In [40]:
#functions to determine the recoil based on the ejectile
def create_recoil(beam,target,ejectile,mass_df):

    mass_number = beam.A + target.A - ejectile.A
    proton_number = beam.Z + target.Z -ejectile.Z

    element = atomicElements[proton_number-1] #-1 because we index at 0

    return Isotope(mass_number,element,mass_df)
    
    

In [41]:
#q-value calculation
def calculate_q_value(beam,target,ejectile,recoil):
    initial_mass = beam.mass + target.mass #in MeV
    final_mass = ejectile.mass + recoil.mass #MeV
    q_val = initial_mass - final_mass #q value is always the same for a given reaction no matter the excitation energy

    recoil.q_value = q_val
    return


In [42]:
def calculate_L_transfer(beam,target,ejectile,recoil):

    parity_change = beam.parity * recoil.parity

    #calculate the two possible L transfers
    L_transfer1 = abs(recoil.spin - beam.spin - (target.spin-ejectile.spin))
    L_transfer2 = abs(recoil.spin - beam.spin + (target.spin-ejectile.spin))

    #now determine which one is needed based on the parity changes
    if (+1 == parity_change):
        if (0 == L_transfer1 % 2):
            recoil.L_transfer = L_transfer1
        else:
            recoil.L_transfer = L_transfer2
    else:
        if (0 == L_transfer1 % 2):
            recoil.L_transfer = L_transfer2
        else:
            recoil.L_transfer = L_transfer1
    

    return

Number of nodes calculations

The number of nodes is given using the Talmi-Moshinsky relationship:

\begin{equation}
    Q=2N+L=\sum_{i}(2n_{i}+l_{i})
\end{equation}

where $Q$ is the oscillator quantum number, $N$ the number of nodes, $L$ the angular momentum transfer, $n_{i}$ the oscillator quantum number of the orbital being transferred into, $l_{i}$ the angular momentum fo the orbital being transferred into, and the sum is taken over all orbitals (in the case of a multi-nucleon transfer). For single neutron transfer, the sum is over just the single orbital being transferred into.



In [44]:
#calculation of number of nodes
def calculate_number_nodes(isotope):

    #calculate Q
    Q = 2*(isotope.orbital_osc_shell) + isotope.orbital_L #-1 because of indexing 1 vs indexing 0

    isotope.nn = (Q-isotope.L_transfer)/2.
    

    return

In [45]:
#function for populating states in the recoiling nucleus
def get_user_excited_state(isotope):

    get_user_excited_state_energy(isotope)
    get_user_spin(isotope)
    get_user_parity(isotope)


    return


In [46]:
#this will define the neutron potential
class NeutronPotential():
    # neutron potential of Wilmore and Hodgson
    # D. Wilmore and P.E. Hodgson, Nuclear Physics 55:673-694 (1964).
    # https://doi.org/10.1016/0029-5582(64)90184-1

    def __init__(self):
        self.A0 = 0.66
        self.AD = 0.48
        self.RC = 0 #no coulomb radius for neutron
        #these all get initalized in InitializeDependentParams()
        self.V0 = 0
        self.R0 = 0
        self.WD = 0
        self.RD = 0

    
    def InitializeDependentParams(self,A,E): #mass number A, lab inverse kinematics energy E
        #convert from lab inverse kinematics lab energy into normal kinematics lab energy
        #energy is Elab (heavy) * (mass light)/(mass heavy)
        #since this is a neutron potential, m(neutron) ~1 amu, so this will be pretty close
        E = E/A 
        #initialize our parameters based on that
        #equations 9-13
        self.V0 = 47.01 - 0.267*E - 0.00118*E**2
        self.R0 = 1.322 - A*7.6e-4 + A**2*4e-6 - A**3*8e-9
        self.WD = max(0,9.52 - 0.053*E)
        self.RD = 1.266 - A*3.7e-4 + A**2*2e-6 - A**3*4e-9

    def __str__(self):

        return f'V0: {self.V0}\nR0: {self.R0}\nA0: {self.A0}\nWD: {self.WD}\nRD: {self.RD}\nAD: {self.AD}'

In [47]:
#proton potential class
class ProtonPotential():


    def __init__(self):
        self.V0 = 0
        self.R0 = 0
        self.A0 = 0
        self.W0 = 0
        self.WD = 0
        self.RD = 0
        self.AD = 0
        self.VLS = 0
        self.RLS = 0
        self.ALS = 0
        self.RC = 0


    def BGPotential(self,A,Z,E): #mass number A, atomic number Z, incident lab energy (of proton beam).
        # this is the potential of Becchetti and Greenlees
        # F.D. Becchetti and G.W. Greenlees, Phys. Rev. 182:1190-1209 (1969)
        # https://doi.org/10.1103/PhysRev.182.1190
    
        #convert from lab inverse kinematics lab energy into normal kinematics lab energy
        #energy is Elab (heavy) * (mass light)/(mass heavy)
        #since this is a proton potential, m(proton) ~1 amu, so this will be pretty close
        E = E/A
        #now initialize everything else
        #equation 8
        self.V0 = 54.0 - 0.32*E + 0.4*Z/(A**(1/3)) + 24.0*(A-2*Z)/A #substituting (A-2*Z) for (N-Z) from the paper
        self.R0 = 1.17
        self.A0 = 0.75
        self.W0 = max(0,0.22*E - 2.7)
        self.WD = max(0,11.8 - 0.25*E + 12.0*(A-2*Z)/A) #substituting (A-2*Z) for (N-Z) from the paper
        self.RD = 1.32
        self.AD = 0.51 + 0.7*(A-2*Z)/A #substituting (A-2*Z) for (N-Z) from the paper
        self.VLS = 6.2
        self.RLS = 1.32
        self.ALS = 0.75
        self.RC = 1.25

    def PPotential(self,A,Z,E):
        # potential of Perey
        # F.G. Perey, Phys. Rev. 131:745-763 (1963)
        # https://doi.org/10.1103/PhysRev.131.745

        #convert from lab inverse kinematics lab energy into normal kinematics lab energy
        #energy is Elab (heavy) * (mass light)/(mass heavy)
        #since this is a proton potential, m(proton) ~1 amu, so this will be pretty close
        E = E/A
        #now initialize everything else
        #equation 2, Table 5, first paragraph in section 3, end of page 753 (possibly equation on 754)
        self.V0 = 53.3 - 0.55*E + (0.4*Z/(A**(1/3)) + 27*(A-2*Z)/A)
        self.R0 = 1.25
        self.A0 = 0.65
        self.W0 = 0
        self.WD = 13.5 #quoted as 13.5+-2.0 MeV
        self.RD = 1.25
        self.AD = 0.47
        self.VLS = 7.5
        self.RLS = 1.25
        self.ALS = 0.47
        self.RC = 1.25
        


    def __str__(self):
        return f'''V0: {self.V0}\nR0: {self.R0}\nA0: {self.A0}\nW0: {self.W0}\nWD: {self.WD}\nRD: {self.RD}\nAD: {self.AD}
            \rVLS: {self.VLS}\nRLS: {self.RLS}\nALS: {self.ALS}\nRC: {self.RC}'''
    

In [48]:
#deuteron potentials here
class DeuteronPotential():


    def __init__(self):
        self.V0 = 0
        self.R0 = 0
        self.A0 = 0
        self.W0 = 0
        self.WD = 0
        self.RD = 0
        self.AD = 0
        self.VLS = 0
        self.RLS = 0
        self.ALS = 0
        self.RC = 0


    def LHPotential(self,A,Z,E):
        # deuteron potential of Lohr and Haeberli
        # J.M. Lohr and W. Haeberli, Nuclear Physics A, 232(2):381-397 (1974)
        # https://doi.org/10.1016/0375-9474(74)90627-7
        
        #convert from lab inverse kinematics lab energy into normal kinematics lab energy
        #energy is Elab (heavy) * (mass light)/(mass heavy)
        #since this is a deuteron potential, m(deuteron) ~2 amu, so this will be pretty close
        E = E*2/A
        #initialize everything else now
        #Eq 1, 2, 3,
        self.V0 = 91.13 + 2.20*Z/(A**(1/3))
        self.R0 = 1.05
        self.A0 = 0.80
        self.W0 = 0
        self.WD = 218*A**(-2/3)
        self.RD = 1.43
        self.AD = 0.50 + 0.013*A**(2/3)
        self.VLS = 7.0
        self.RLS = 0.75
        self.ALS = 0.5
        self.RC = 1.3



    def DPotential(self,A,Z,E):
        # potential of Daehnick
        # W.W. Daehnick, J.D. Childs, Z. Vrcelj, Phys. Rev. C 21:2253-2274 (1980)
        # https://doi.org/10.1103/PhysRevC.21.2253

        #convert from lab inverse kinematics lab energy into normal kinematics lab energy
        #energy is Elab (heavy) * (mass light)/(mass heavy)
        #since this is a deuteron potential, m(deuteron) ~2 amu, so this will be pretty close
        E = E*2/A
        #initialize everything else now
        #Table 3, Potential "79 DCV, F"
        self.V0 = 88.0 - 0.283*E + 0.88*Z*A**(-1/3) #also called V_R sometimes
        self.R0 = 1.17
        self.A0 = 0.717 + 0.0012*E
        self.W0 = (12 + 0.031*E)*(1-np.exp(-(E/100)**2))
        self.WD = (12 + 0.031*E)*np.exp(-(E/100)**2)
        self.RD = 1.376 - 0.01*np.sqrt(E)
        self.AD = 0.52 + 0.07*A**(1/3) #this ignores the contributions from the magic number sum in this potential for now
        self.VLS = 7.20 - 0.032*E
        self.RLS = 1.07
        self.ALS = 0.66
        self.RC = 1.30

    def __str__(self):
        return f'''V0: {self.V0}\nR0: {self.R0}\nA0: {self.A0}\nW0: {self.W0}\nWD: {self.WD}\nRD: {self.RD}\nAD: {self.AD}
            \rVLS: {self.VLS}\nRLS: {self.RLS}\nALS: {self.ALS}\nRC: {self.RC}'''


In [49]:
#determine the potentials to use:
def determine_neutron_potential():
    #no option right now. Just return the potential

    return "WH" #Wilmore and Hodgson potential

def determine_proton_potential():
    #BG or Perey potentials. Default is "BG"
    possible_potentials = ["BG","P"]

    print("Possible proton potentials:")
    for p in possible_potentials:
        print(p)
    
    user_pot = input("Enter the proton potential (default BG): ")

    if user_pot not in possible_potentials:
        user_pot = 'BG'

    return user_pot

def determine_deuteron_potential():
    #LH or Daehnick potentials. Default is LH
    possible_potentials = ["LH","Daehnick"]

    print("Possible deuteron potentials:")
    for p in possible_potentials:
        print(p)

    user_pot = input("Enter the deuteron potential (default LH): ")

    if user_pot not in possible_potentials:
        user_pot = 'LH'

    return user_pot

In [50]:
#construction of the file in the function here

def Fresco_constructor():

    #First, get the beam that we want for the (d,p) reaction:    
    beam_isotope = get_user_isotope(masses) # Isotope(user_isotope[0],user_isotope[1])
    
    #get our initial spin-parity values
    get_user_spin_parity(beam_isotope)

    #get beam energy
    user_energy = get_user_beam_energy()
    
    #if a reaction other than (d,p) is wanted, these next sections shouuld be edited.
    
    #we also need to get the deuteron
    target = Isotope(2,'H',masses)
    target.spin = 1
    target.parity = 1

    #define our ejectile (proton)
    ejectile = Isotope(1,'H',masses)
    ejectile.spin = 0.5
    ejectile.parity = 1
    
    
    #calculate our binding energy of the target
    target.binding_energy = calculate_deuteron_binding_energy(masses)

    
    #create the states in the final nucleus here
    num_states = get_user_number_states()
    final_states = []
    for _ in range(num_states):
         #calculate our recoil
        recoil = create_recoil(beam_isotope,target,ejectile,masses)
        get_user_excited_state(recoil)
        calculate_q_value(beam_isotope,target,ejectile,recoil)
        calculate_L_transfer(beam_isotope,target,ejectile,recoil)

        transferred_part = calculate_transferred_particle(beam_isotope,recoil,masses)
        
        calculate_binding_energy(recoil,transferred_part,masses)
        get_user_orbital(recoil)
        calculate_number_nodes(recoil)
        final_states.append(recoil)


    #starting construction of the FRESCO file here
    #this is the part that is not read by FRESCO, it is for the user's benefit
    fresco_string = f' {beam_isotope.isotope}(d,p){recoil.isotope} at {user_energy} MeV in inverse kinematics'

    #add the header that starts the FRESCO initialization
    fresco_string += '\n' + construct_fresco_header(user_energy)
    #add the first partition for the beam and target
    fresco_string += '\n\n' + construct_partition(beam_isotope,target,beam_isotope,1)#only one state for the beam and target
    #we use beam_isotope both times so that the q-value is 0 for the beam (which it should be because there is no reaction)

    #construct the intial state for the beam+target
    fresco_string+='\n' + construct_states(beam_isotope,target,recoil)

    #now construct the final states
    fresco_string +='\n\n'+construct_partition(recoil,ejectile,recoil,num_states)
    for i in range(num_states):
        if (i > 0):
            fresco_string += '\n '
        else:
            fresco_string += '\n'
            
        fresco_string +=construct_states(final_states[i],ejectile,final_states[i])
   
    fresco_string += '\n&partition /'
    #end of partition here

    
    #potentials here

    #initialize our potentials first:
    neut_pot = NeutronPotential()
    neut_pot.InitializeDependentParams(beam_isotope.A,user_energy)

    prot_pot = ProtonPotential() #beam with proton
    deut_pot = DeuteronPotential() #beam with deuteron
    prot_recoil_pot = ProtonPotential() #recoil with proton

    #determine what potentials to use
    determine_neutron_potential() #currently doesn't do anything.

    #proton:
    ppot = determine_proton_potential()
    if ('P' == ppot):
        prot_pot.PPotential(beam_isotope.A,beam_isotope.Z,user_energy)
        prot_recoil_pot.PPotential(recoil.A,recoil.Z,user_energy)
    else:
        prot_pot.BGPotential(beam_isotope.A,beam_isotope.Z,user_energy)
        prot_recoil_pot.BGPotential(recoil.A,recoil.Z,user_energy)
    #deuteron:
    dpot = determine_deuteron_potential()
    if ('Daehnick' == dpot):
        deut_pot.DPotential(beam_isotope.A,beam_isotope.Z,user_energy)
        
    else:
        deut_pot.LHPotential(beam_isotope.A,beam_isotope.Z,user_energy)
    
    #beam+deuteron potential
    fresco_string += '\n\n' + construct_potential(beam_isotope,deut_pot,user_energy,1)

    #beam+proton:
    fresco_string += '\n\n' + construct_potential(beam_isotope,prot_pot,user_energy,2)

    #beam+neutron:
    fresco_string += '\n\n' + construct_potential(beam_isotope,neut_pot,user_energy,3)

    #standard deuteron potential
    fresco_string += '\n\n' + construct_deuteron_potential()

    #recoil+proton:
    fresco_string += '\n\n' + construct_potential(recoil,prot_recoil_pot,user_energy,5)

    fresco_string += '\n\n &pot /'
    #end of potentials here

    
    #overlap here

    #overlap of deuteron first:
    fresco_string += '\n\n\n' + construct_overlap(target,"")

    for i in range(len(final_states)):
        fresco_string += '\n  ' + construct_overlap(final_states[i],str(i+1))

    fresco_string +='\n &overlap /'
    #end of overlap
    
    #coupling info here
    #coupling header
    fresco_string +='\n\n' + construct_coupling()
    
    #deuteron coupling
    fresco_string += "\n" + construct_cfp("")
    #other state couplings
    for i in range(len(final_states)):
        fresco_string += '\n ' + construct_cfp(str(i+1)) + ' ' 
    

    fresco_string +=  '\n\n&cfp /\n\n&coupling\n\n'

    #end of coupling


    #end of construction of file. Save it now
    file_name = f'{beam_isotope.isotope}_dp_{recoil.isotope}_{dpot}_{ppot}_{int(user_energy)}_MeV.in'
    out_file = open(file_name,'w')
    out_file.write(fresco_string)
    out_file.close()


    
    print(f'\nFile written as {file_name}')

    print('\n\nEnd of program!\n\n')


In [51]:
Fresco_constructor()

Please enter the name (ex: Kr): Kr
Please enter the mass number:  86


Even-even nucleus. Using spin 0+


Enter total beam energy (MeV):  688
Enter the number of states to populate:  1
Enter the excited-state energy (MeV):  0
Enter the spin:  2.5
Please enter the parity (±1):  1
Enter the orbital (ex: 2d5/2):  2d5/2


Possible proton potentials:
BG
P


Enter the proton potential (default BG):  


Possible deuteron potentials:
LH
Daehnick


Enter the deuteron potential (default LH):  



File written as 86Kr_dp_87Kr_LH_BG_688_MeV.in


End of program!


